# Multiplicity determination with Calibr

Note: Uses the R3BRoot NeuLAND module. As this module is compiled with ROOT, it is available in Python by `import ROOT` if the environment variables are set.

## Example Fit / Calibration
We can run the full calibration from python, with some amount of yucky syntax

In [1]:
labels = ["nPN", "nPH"]
neutrons = [2, 3, 4, 5]
doubleplanes = [12, 30]

In [2]:
import joblib
from train_calibr import train

joblib.Parallel(n_jobs=-1, backend="loky", verbose=1)(
    joblib.delayed(train)(
        doubleplane=doubleplane,
        energy=600,
        nmax=nmax,
        physics="inclxx",
        label=label,
    )
    for doubleplane in doubleplanes
    for label in labels
    for nmax in neutrons
)

print("Done")

Welcome to JupyROOT 6.16/00


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  16 | elapsed:  1.3min remaining:  3.9min
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:  3.8min finished


Done


## Example Prediction

In [3]:
import os
import sys

import numpy as np
import pandas as pd
import ROOT
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, plot_confusion_matrix

sys.path.append("..")
from helpers import filename_for

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [4]:
max_neutrons = 4

for dp in doubleplanes:
    dfs = [pd.read_pickle(filename_for(15, dp, 600, 500, neutrons, "inclxx", subrun, "trifeature.pkl")) for subrun in range(20) for neutrons in range(1, max_neutrons + 1)]
    testdata = pd.concat(dfs, ignore_index=True)
    # traindata.loc[traindata["nHits"] == 0, "nPN"] = 0

    for label in labels:
        cutf = ROOT.TFile.Open(f"models/{dp}dp_600AMeV_{max_neutrons}n_{label}.ncut.root")
        para = cutf.Get("R3BNeulandMultiplicityCalorimetricPar")

        y_true = testdata[[label]].values.ravel()
        y_pred = np.array([para.GetNeutronMultiplicity(row["Edep"], row["nClus"]) for _, row in testdata.iterrows()])

        bac = balanced_accuracy_score(y_true, y_pred)
        print(dp, label, bac)
        # cm = confusion_matrix(y_true, y_pred, labels=range(0, max_neutrons + 1))
        # print(cm)
        # cmrel = confusion_matrix(y_true, y_pred, labels=range(0, max_neutrons + 1), normalize="true")
        # print(cmrel)

        cutf.Close()

/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


12 nPN 0.41278624999999997
12 nPH 0.6567866071425283


/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


30 nPN 0.6978550000000001
30 nPH 0.819154348227805
